# Problem 1

I decided to use a custom problem I named "FactoryTaxEmissionsProblem" in which 10 factories are taxed on their output and are regulated by emission laws. The emission laws place constraints on how much product two neighboring factories can produce. For example, one such law might be that factories 1 and 2 can only produce 1600 combined units due to the emissions produced as byproducts. The goal of the problem is to determine the optimal number of units that each factory should produce given some emission constraints and tax values. The cost function represents the profit from the ten factories, where the number of units produced by each factory is multiplied by some modifier to represent the post-tax value. 


Formally, the cost function is defined as the following:
$$C(\bf{x}) = \bf{c}^{T}\bf{x}$$

Where:

$$ \bf{c} = \left\lgroup \matrix{.8 \cr 1 \cr .95 \cr .72 \cr .9 \cr .87 \cr .72 \cr .82 \cr .93 \cr .67} \right\rgroup $$

$$ \bf{x} = \left\lgroup \matrix{x_{1} \cr x_{2} \cr x_{3} \cr x_{4} \cr x_{5} \cr x_{6} \cr x_{7} \cr x_{8} \cr x_{9} \cr x_{10}} \right\rgroup $$

The constraints on the variables on the factory emissions:

$$ x_{1} \le 1000 $$
$$ x_{2} \le 1000 $$
$$ x_{3} \le 3000 $$
$$ x_{4} \le 1500 $$
$$ x_{5} \le 1600 $$
$$ x_{6} \le 750 $$
$$ x_{7} \le 2000 $$
$$ x_{8} \le 1100 $$
$$ x_{9} \le 500 $$
$$ x_{10} \le 350 $$
$$ x_{1} + x_{2} \le 1500 $$
$$ x_{6} + x_{7} + x_{8} = 1800 $$

I represented the constrained version of the problem using the Python class ConstrainedFactoryTaxProblem below using matrices for the equality contraints and for the inequality constrained as required by the scipy library.

In [5]:
import numpy

FactoryCostCoefficients = [.8, 1, .95, .72, .9, .87, .72, .82, .93, .67]
FactoryConstraints = [
        1000, 1000, 3000, 1500, 1600, 750, 2000, 1100, 500, 350, 1500
    ]


class ConstrainedFactoryTaxProblem:
    c = numpy.asarray(FactoryCostCoefficients) * -1
    A_u = numpy.eye(10)
    A_u = numpy.append(A_u, [[1, 1] + [0] * 8], axis=0)
    b_u = FactoryConstraints

    A_eq = numpy.zeros((10, 10))
    A_eq[0] = [0] * 5 + [1, 1, 1, 0, 0]

    b_eq = [1800] + [0] * 9

I tested two versions of the scipy **optimize.linprog()** method and the **scipy.minimize()** method (unconstrained). The two constrained scipy methods were the simplex and interior-point methods. They were tested using the following Python code.

In [8]:
import stats_generator
import scipy.optimize
import time

def test_scipy_method(method, iters):
    test_results = []
    for i in range(iters):
        start = time.time()
        res = scipy.optimize.linprog(c=ConstrainedFactoryTaxProblem.c, A_eq=ConstrainedFactoryTaxProblem.A_eq, b_eq=ConstrainedFactoryTaxProblem.b_eq,
                                     A_ub=ConstrainedFactoryTaxProblem.A_u, b_ub=ConstrainedFactoryTaxProblem.b_u, method=method)
        end = time.time()
        test_results.append((res.fun, (end-start) * 1000))

    func_vals = list(map(lambda x: x[0], test_results))
    time_vals = list(map(lambda x: x[1], test_results))

    return ((stats_generator.StatsGenerator.unbiased_expected_value(func_vals), stats_generator.StatsGenerator.std_dev(func_vals)),
            (stats_generator.StatsGenerator.unbiased_expected_value(time_vals), stats_generator.StatsGenerator.std_dev((time_vals))))

def print_stats(header, stats):
    print(header)
    print("Avg. function value: ", stats_generator.StatsGenerator.stats_to_string(stats[0]))
    print("Avg. time value (ms): ", stats_generator.StatsGenerator.stats_to_string(stats[1]))
    print("\n")
    
stats = test_scipy_method("simplex", 1000)
print_stats("SCIPY SIMPLEX METHOD", stats)

stats = test_scipy_method("interior-point", 1000)
print_stats("SCIPY INTERIOR-POINT METHOD", stats)

SCIPY SIMPLEX METHOD
Avg. function value:  -8.98e+03±0.00e+00
Avg. time value (ms):  3.95e+00±4.89e-01


SCIPY INTERIOR-POINT METHOD
Avg. function value:  -8.98e+03±2.24e-10
Avg. time value (ms):  4.66e+00±2.60e-01




The simplex method achieved a more accuracte optimal value in a shorter amount of time compared to the interior-point method.